# **_PyTorch Basics_**

#### _https://www.youtube.com/playlist?list=PLqnslRFeH2UrcDBWF5mfPGpqQDSta6VK4_

#### __*Call the Libraries & GPU*__

In [ ]:
import math
import torch
import torchvision                                   # 1st use: Dataset & Dataloader
import torch.nn as nn                                # 1st use: Training Pipeline
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets                         # 1st use: Linear Regression
from sklearn.preprocessing import StandardScaler     # 1st use: Logistic Regression
from sklearn.model_selection import train_test_split # 1st use: Logistic Regression
from torch.utils.data import Dataset, DataLoader     # 1st use: Dataset & Dataloader

GPU = torch.device("cpu")
if torch.cuda.is_available():
    GPU = torch.device("cuda")

print("Using CPU") if GPU == torch.device("cpu") else print("Using GPU")

#### __*Create a Tensor and basic operations*__

In [ ]:
x = torch.tensor([[1, 1],[1, 1]], dtype=torch.float64)
print(x)
print(x.dtype)
y = torch.rand(2, 2)
print(y)
print(f'{y.dtype}\n')
#z = x + y
#print(z)

#y = y.add(x)
#y = torch.add(x, y) # add(+), sub(-), mul(*), div(/)
y.add_(x) # in-place operation (same as line 10)
print(y)

#### __*Select and view certain stuff*__

In [ ]:
x = torch.rand(5, 4)
print(x, x.size())
print(x[:, 0]) # print 1st column
print(x[1, :]) # print 2nd row
print(x[0, 0])
print(x[0, 0].item()) # print a direct value

y = x.view(5*4) # re-shapes the tensor into 1D
print(y, y.size())

y = x.view(-1, 10) # calculate the other value
print(y, y.size())

#### __*Convert from torch to numpy and vice-versa*__

In [ ]:
x = torch.rand(5)
print(x, type(x))
y = x.numpy() # y shares the location with x, changing x will change y as well
print(f'{y} {type(y)}\n')

x = np.ones(5)
print(x, type(x))
y = torch.from_numpy(x) # y shares the location with x, changing x will change y as well
print(y, type(y))

#### __*Create a Tensor on the GPU*__

In [ ]:
x = torch.rand(5, device=GPU) # cannot be converted to numpy
y = torch.rand(5)
y = y.to(GPU)
z = x + y
z = z.to("cpu")

try:
    x = x.numpy()
except TypeError:
    print("Cannot covert to GPU Tensor Numpy")

try:
    z = z.numpy()
    print(z, type(z))
except TypeError:
    print("Cannot covert to GPU Tensor Numpy")

#### __*AutoGrad*__

In [ ]:
x = torch.randn(3, requires_grad=True)
print(x)
# ⌄⌄⌄ These methods will remove grad
# x.requires_grad_(False)
# x.detach_()
# with torch.no_grad():
#   y = x + 2

y = x + 2
print(y)

z = y * y * 2
z = z.mean()
print(z)

z.backward() # dz/dx
print(x.grad)

#### __*Demo Dummy Function*__

In [ ]:
weights = torch.randn(4, requires_grad=True)

for epoch in range(3):
    model_output = (weights * 3).sum()
    model_output.backward()
    print(weights.grad)

    weights.grad.zero_() # we have to clear the grad

#### __*Backpropagation*__

In [ ]:
x = torch.tensor(1.0)
y = torch.tensor(2.0)

w = torch.tensor(1.0, requires_grad=True)

# forward pass and compute the loss
y_hat = w * x
loss = (y_hat - y)**2
print(loss)

# backward pass
loss.backward()
print(w.grad)

### update weights
### next forward and backward

#### __*Manual Gradient Descent*__

In [ ]:
# f = w * x
# f = 2 * x

X = np.array([1, 2, 3, 4], dtype=np.float32)
Y = np.array([2, 4, 6, 8], dtype=np.float32)

w = 0.0

# model prediction
def forward(x):
    return w * x

# loss = MSE
def loss(y, y_predicted):
    return ((y_predicted - y)**2).mean()

# gradient
# MSE = 1/N * (w*x - y)**2
# dJ/dw = 1/N * 2x * (w*x - y)
def gradiant(x, y, y_predicted):
    return np.dot(2*x, y_predicted - y).mean()

print(f'Prediction before training: f(5) = {forward(5):.3f}')

# Training
learning_rate = 0.01
n_iters = 20

for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = forward(X)

    #loss
    L = loss(Y, y_pred)

    # gradients
    dw = gradiant(X, Y, y_pred)

    # update weights
    w -= learning_rate * dw

    if epoch % 2 == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss = {L:.8f}')

print(f'Prediction after training: f(5) = {forward(5):.3f}')

#### __*Automatic Gradient Descent*__

In [ ]:
# f = w * x
# f = 2 * x

X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

# model prediction
def forward(x):
    return w * x

# loss = MSE
def loss(y, y_predicted):
    return ((y_predicted - y)**2).mean()

print(f'Prediction before training: f(5) = {forward(5):.3f}')

# Training
learning_rate = 0.01
n_iters = 80

for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = forward(X)

    #loss
    L = loss(Y, y_pred)

    # gradients = backward pass
    L.backward() # dL/dw

    # update weights
    with torch.no_grad():
        w -= learning_rate * w.grad

    # !!! ZERO the gradiants !!!
    w.grad.zero_()

    if epoch % int(n_iters/10) == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss = {L:.8f}')

print(f'Prediction after training: f(5) = {forward(5):.3f}')

#### __*Training Pipeline (Manual Prediction)*__

In [ ]:
# 1.) Design model (input, output size, forward pass)
# 2.) Construct loss and optimizer
# 3.) Training loop
#       - forward pass: compute prediction
#       - backward pass: gradients
#       - update weights

# f = w * x
# f = 2 * x

X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

# model prediction
def forward(x):
    return w * x

print(f'Prediction before training: f(5) = {forward(5):.3f}')

# Training
learning_rate = 0.01
n_iters = 80

loss = nn.MSELoss()
optimizer = torch.optim.SGD([w], lr=learning_rate)

for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = forward(X)

    #loss
    L = loss(Y, y_pred)

    # gradients = backward pass
    L.backward() # dL/dw

    # update weights
    optimizer.step()

    # !!! ZERO the gradiants !!!
    optimizer.zero_grad()

    if epoch % int(n_iters/10) == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss = {L:.8f}')

print(f'Prediction after training: f(5) = {forward(5):.3f}')

#### __*Training Pipeline (Automatic Prediction)*__

In [ ]:
# 1.) Design model (input, output size, forward pass)
# 2.) Construct loss and optimizer
# 3.) Training loop
#       - forward pass: compute prediction
#       - backward pass: gradients
#       - update weights

# f = w * x
# f = 2 * x

X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32) #, device=GPU)
Y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32) #, device=GPU)

X_test = torch.tensor([5], dtype=torch.float32) #, device=GPU)

n_samples, n_features = X.shape
print(n_samples, n_features)

input_size = n_features
output_size = n_features

#model = nn.Linear(input_size, output_size) #, device=GPU)
class LinearRegression(nn.Module):

    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define layers
        self.lin = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.lin(x)

model = LinearRegression(input_size, output_size)

print(f'Prediction before training: f(5) = {model(X_test).item():.3f}')

# Training
learning_rate = 0.001
n_iters = 30000

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = model(X)

    #loss
    L = loss(Y, y_pred)

    # gradients = backward pass
    L.backward() # dL/dw

    # update weights
    optimizer.step()

    # !!! ZERO the gradiants !!!
    optimizer.zero_grad()

    if epoch % int(n_iters/10) == 0:
        [w, b] = model.parameters()
        print(f'epoch {epoch+1}: w = {w[0][0].item():.3f}, loss = {L:.8f}')

print(f'Prediction after training: f(5) = {model(X_test).item():.3f}')

#### __*Linear Regression*__

In [ ]:
# 0.) Data Preparation
X_numpy, y_numpy = datasets.make_regression(n_samples=100, n_features=1, noise=20, random_state=1)

X = torch.from_numpy(X_numpy.astype(np.float32))
y = torch.from_numpy(y_numpy.astype(np.float32))
y = y.view(y.shape[0], 1)

n_samples, n_features = X.shape

# 1.) Model
input_size = n_features
output_size = 1
model = nn.Linear(input_size, output_size)

# 2.) Loss & Optimizer
learning_rate = 0.01
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3.) Training Loop
num_epochs = 250
for epoch in range(num_epochs):
    # Forward Pass & Loss
    y_predicted = model(X)
    L = criterion(y_predicted, y)
    # Backward Pass
    L.backward()
    #Update
    optimizer.step()
    #!!! Zero Gradiants !!!
    optimizer.zero_grad()

    if epoch % int(num_epochs/10) == 0:
        print(f'epoch {epoch+1}: loss = {L.item():.8f}')

# Plot
predicted = model(X).detach().numpy()
plt.plot(X_numpy, y_numpy, 'ro')
plt.plot(X_numpy, predicted, 'b')
plt.show()

#### __*Logistic Regression*__

In [ ]:
# 0.) Data Preparation
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Scale
sc = StandardScaler() # recommended with Logistic Regression
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
#X_train = X_train.to(GPU)
X_test = torch.from_numpy(X_test.astype(np.float32))
#X_test = X_test.to(GPU)
y_train = torch.from_numpy(y_train.astype(np.float32))
#y_train = y_train.to(GPU)
y_test = torch.from_numpy(y_test.astype(np.float32))
#y_test = y_test.to(GPU)

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

# 1.) Model
# f = wx + b, Sigmoid at the End
class LogisticRegression(nn.Module):

    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

model = LogisticRegression(n_features)
#model = model.to(GPU)

# 2.) Loss & Optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3.) Training Loop
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward Pass & Loss
    y_predicted = model(X_train)
    L = criterion(y_predicted, y_train)
    # Backward Pass
    L.backward()
    #Update
    optimizer.step()
    #!!! Zero Gradiants !!!
    optimizer.zero_grad()

    if epoch % int(num_epochs/10) == 0:
        print(f'epoch {epoch+1}: loss = {L.item():.8f}')

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'Accuracy = {acc:.4f}')

#### __*Dataset & Dataloader*__
<div class="alert alert-block alert-success">

__epoch = 1 forward & backward pass of ALL training samples__

__batch_size = number of training samples in one forward & backward pass__

__number of iterations = number of passes, each pass using [batch_size] number of samples__

__e.g. 100 samples, batch_size=20 --> 100/20 = 5 iterations per 1 epoch?__
</div>

In [ ]:
class WineDataset(Dataset):

    def __init__(self):
        # Load Data
        xy = np.loadtxt('D:\GitHub\CNN\datasets\wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.x = torch.from_numpy(xy[:, 1:])
        self.y = torch.from_numpy(xy[:, [0]]) # n_samples, 1
        self.n_samples = xy.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples

WineDataset = WineDataset()

first_data = WineDataset[0]
features, labels = first_data
print(features, labels)

#### __*Dataset Transforms*__

_https://pytorch.org/vision/stable/transforms.html_

In [ ]:
class WineDataset(Dataset):

    def __init__(self, transform=None):
        # Load Data
        xy = np.loadtxt('D:\GitHub\CNN\datasets\wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]

        self.x = xy[:, 1:]
        self.y = xy[:, [0]]

        self.transform = transform

    def __getitem__(self, index):
        sample = self.x[index], self.y[index]

        if self.transform:
            sample = self.transform(sample)

        return sample

    def __len__(self):
        return self.n_samples


class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)


class MulTransform:
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets


WineDatasetNumpy = WineDataset(transform=None)
first_data = WineDatasetNumpy[0]
features, labels = first_data
print(type(features), type(labels))
print(f'{features}\n')

WineDatasetTensor = WineDataset(transform=ToTensor())
first_data = WineDatasetTensor[0]
features, labels = first_data
print(type(features), type(labels))
print(f'{features}\n')

# Combine more tranformations
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
WineDatasetComposed = WineDataset(transform=composed)
first_data = WineDatasetComposed[0]
features, labels = first_data
print(type(features), type(labels))
print(features)

#### __*Softmax*__

In [ ]:
def softmax(x):
    return (np.exp(x) / np.sum(np.exp(x), axis=0))

x = np.array([2.0, 1.0, 0.1])
outputs = softmax(x)
print(f'Softmax Numpy: {outputs}')

x = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(x, dim=0)
print(f'Softmax Torch: {outputs}')